In [41]:
import duckdb
import pandas as pd
import numpy as np
import re
import unidecode

In [3]:
DATABASE_PATH = "../data/staging/data.duckdb"

db = duckdb.connect(DATABASE_PATH)

In [13]:
profiles = db.sql("SELECT * FROM profiles").df()
profiles.head(n=10)

,id,ad_id,profile_type,name,religion,age,sex,country_of_residence,country_of_origin,region_of_origin,village_of_origin,sector_of_activity,marital_status,has_children,number_of_children,qualities,values,defects,interests,height,weight,physical_appearance,economic_situation,education_level,illness,relationship_goal,created_at
0,2,1,ADVERTISER,None,None,35 ans,Femme,[Canada],C,None,None,Début de carrière,Célibataire,False,sans enfant,"[respectueuse, douce, travailleuse]",[],[],[],1.70 m,70 kg,None,Début de carrière,None,None,fonder une famille solide,2025-10-06 03:26:36.384601
1,3,1,DESIRED,None,None,35-42 ans,Homme,[Canada],None,None,None,None,None,False,sans enfant,"[travailleur, respectueux, sens de l'humour]",[],[],[],1.72 m,None,None,None,None,None,fonder une famille solide,2025-10-06 03:26:36.393523
2,4,2,ADVERTISER,None,None,39 ans,Femme,[France],None,None,None,None,None,True,1 enfant,"[positive, intègre, joie de vivre contagieuse]","[Générosité, respect, tolérance, famille, comm...",<NA>,"[voyages, développement personnel, plaisirs si...",1.68 m,67 kg,charmante,None,None,None,None,2025-10-06 03:26:36.395633
3,5,2,DESIRED,None,None,37-45 ans,Homme,"[France, IDF]",None,None,None,None,None,False,None,"[charmant, confiance en toi, attentionné, conc...",[],<NA>,"[sportif, courir]",1.77 m,None,None,None,None,None,None,2025-10-06 03:26:36.398146
4,6,3,ADVERTISER,None,None,38 ans,Femme,[Canada],C,O,None,None,célibataire,True,3 enfants,"[respectueuse, gentille, responsable, travaill...","[respect, vérité, amour, communication, valeur...",[],"[danser, voyager, sorties en couple, sorties e...",None,None,None,None,None,None,None,2025-10-06 03:26:36.400835
5,7,3,DESIRED,None,None,40-55 ans,Homme,[Canada],None,None,None,None,None,False,None,"[aimant, responsable, honnête, sincère, travai...","[respect, vérité, amour, communication, valeur...",[],[],None,None,None,None,None,None,None,2025-10-06 03:26:36.403198
6,8,4,ADVERTISER,None,None,30 ans,Masculin,[France],C,R,None,None,None,False,None,[],[],[],[],1.77 m,78 kg,None,stable,None,None,None,2025-10-06 03:26:36.405451
7,9,4,DESIRED,None,None,24-27 ans,Femme,[France],C,None,None,None,None,False,None,[pragmatique],[bilatéral],[],[],None,None,None,None,None,None,None,2025-10-06 03:26:36.407428
8,10,5,ADVERTISER,None,None,27 ans,Femme,[Canada],C,O,None,None,None,False,sans enfant,"[posé, travailleuse, pas trop sociable, génére...",[],[],"[tourisme, lire, faire du sport, cuisiner]",1.75 m,None,corpulence moyenne avec des formes là où il faut,None,None,None,None,2025-10-06 03:26:36.409458
9,11,5,DESIRED,None,None,30-39 ans,Homme,"[Canada, USA, Europe, Cameroun]",None,None,None,None,None,False,0-1 enfant,"[patient, intelligent, aimable, responsable, h...",[humaines et familiales],[],[],1.80 m,None,None,None,None,None,None,2025-10-06 03:26:36.411540


In [237]:
db.close()

## Age

In [15]:
profiles['age'].unique()

array(['35 ans', '35-42 ans', '39 ans', '37-45 ans', '38 ans',
       '40-55 ans', '30 ans', '24-27 ans', '27 ans', '30-39 ans',
       '33 ans', '33 ans et plus', None, '29 ans', '21-27 ans',
       'max 32 ans', '34 ans', '24-31 ans', '38-50 ans', '31-36 ans',
       '25-30 ans', '46 ans', '46-50 ans', '28-32 ans', '32 ans',
       'max 30 ans', 'max 25 ans', '23-27 ans', '26-31 ans', '30-34 ans',
       '38-45 ans', '31 ans', 'max 27 ans', '28 ans', '19-24 ans',
       '18-24 ans', '40 ans', '27-35 ans', '24 ans', '23-30 ans',
       '30-45 ans', '20-26 ans', 'max 26 ans', '30 ans ou plus',
       '40-49 ans', '26 ans', '28-33 ans', '36 ans', '38 ans max',
       '36-45 ans', '39-45 ans', 'max 31 ans', '34-45 ans', '28-34 ans',
       '23-32 ans', '35-40 ans', '29 ans et plus', '20-27 ans', '25 ans',
       '20-25 ans', '29-33 ans', '30-40 ans', '28-35 ans', '19-25 ans',
       '18 ans et plus', '18-22 ans', '21-35 ans', '33-42 ans',
       '21-25 ans', '20-23 ans', '22-28 ans', '22

In [27]:
def parse_age(age_str):
    """Convertit une chaîne d'âge en (age_min, age_max, age_median)."""
    if not age_str or not isinstance(age_str, str):
        return (np.nan, np.nan, np.nan)
    
    age_str = age_str.strip().lower()

    # "plus de 50 ans" (forme non gérée avant)
    if re.match(r"plus de\s*\d{1,2}", age_str):
        a = int(re.search(r"\d{1,2}", age_str).group())
        return (a, 99, a + (99 - a) / 2)

    # "moins de X ans"
    if re.match(r"moins de\s*\d{1,2}", age_str):
        a = int(re.search(r"\d{1,2}", age_str).group())
        return (0, a, a / 2)

    # 1️⃣ Forme "X ans" (un seul âge)
    single_age = re.match(r"^(\d{1,2})\s*ans?$", age_str)
    if single_age:
        age = int(single_age.group(1))
        return (age, age, age)

    # 2️⃣ Forme "X-Y ans" ou "X à Y ans"
    range_age = re.match(r"^(\d{1,2})\s*(?:-|à|–)\s*(\d{1,2})", age_str)
    if range_age:
        a1, a2 = int(range_age.group(1)), int(range_age.group(2))
        return (a1, a2, (a1 + a2) / 2)

    # 3️⃣ Forme "max X ans" ou "X ans max"
    max_age = re.match(r"^(?:max\s*)?(\d{1,2})\s*ans\s*(?:max)?$", age_str)
    if max_age:
        a = int(max_age.group(1))
        return (0, a, a / 2)

    # 4️⃣ Forme "min X ans"
    min_age = re.match(r"^(?:min\s*)?(\d{1,2})\s*ans$", age_str)
    if min_age and "min" in age_str:
        a = int(min_age.group(1))
        return (a, 99, a + (99 - a) / 2)

    # 5️⃣ Forme "moins de X ans"
    less_age = re.match(r"moins de\s*(\d{1,2})", age_str)
    if less_age:
        a = int(less_age.group(1))
        return (0, a, a / 2)

    # 6️⃣ Forme "plus de X ans" / "X ans et plus" / "X ans ou plus"
    more_age = re.match(r"(\d{1,2})\s*ans?\s*(et|ou)?\s*plus", age_str)
    if more_age:
        a = int(more_age.group(1))
        return (a, 99, a + (99 - a) / 2)

    # 7️⃣ Autres formes "20-35" sans "ans"
    range_simple = re.match(r"(\d{1,2})\s*[-à]\s*(\d{1,2})", age_str)
    if range_simple:
        a1, a2 = int(range_simple.group(1)), int(range_simple.group(2))
        return (a1, a2, (a1 + a2) / 2)

    return (np.nan, np.nan, np.nan)


In [28]:
ages = [(a, ) + parse_age(a) for a in profiles['age'].unique()]
for a in ages:
    print(a)

('35 ans', 35, 35, 35)
('35-42 ans', 35, 42, 38.5)
('39 ans', 39, 39, 39)
('37-45 ans', 37, 45, 41.0)
('38 ans', 38, 38, 38)
('40-55 ans', 40, 55, 47.5)
('30 ans', 30, 30, 30)
('24-27 ans', 24, 27, 25.5)
('27 ans', 27, 27, 27)
('30-39 ans', 30, 39, 34.5)
('33 ans', 33, 33, 33)
('33 ans et plus', 33, 99, 66.0)
(None, nan, nan, nan)
('29 ans', 29, 29, 29)
('21-27 ans', 21, 27, 24.0)
('max 32 ans', 0, 32, 16.0)
('34 ans', 34, 34, 34)
('24-31 ans', 24, 31, 27.5)
('38-50 ans', 38, 50, 44.0)
('31-36 ans', 31, 36, 33.5)
('25-30 ans', 25, 30, 27.5)
('46 ans', 46, 46, 46)
('46-50 ans', 46, 50, 48.0)
('28-32 ans', 28, 32, 30.0)
('32 ans', 32, 32, 32)
('max 30 ans', 0, 30, 15.0)
('max 25 ans', 0, 25, 12.5)
('23-27 ans', 23, 27, 25.0)
('26-31 ans', 26, 31, 28.5)
('30-34 ans', 30, 34, 32.0)
('38-45 ans', 38, 45, 41.5)
('31 ans', 31, 31, 31)
('max 27 ans', 0, 27, 13.5)
('28 ans', 28, 28, 28)
('19-24 ans', 19, 24, 21.5)
('18-24 ans', 18, 24, 21.0)
('40 ans', 40, 40, 40)
('27-35 ans', 27, 35, 31.0)
('

33 et plus => min 33, mean 33

max 32 ans => min l'age de l'annonceur et max 32, si l'annonceur est une femme sinon min c'est l'age minimum legal pour une femme dans le pays de l'annonceur

29 ans et plus => si l'annonceur est une femme alors min 29, max 50 ans (en supposant qu'une femme n'accepterait pas ce marier avec quelqu'un de plus de 20 ans son ainee). si l'annonceur est un homme alors c'est min 29 ans, max son age


## Sex

In [29]:
profiles['sex'].unique()

array(['Femme', 'Homme', 'Masculin', None, 'Fille', 'F', 'M', 'H',
       'Féminin', 'Feminin', 'Mâle'], dtype=object)

In [30]:
def normalize_sex(value: str) -> str:
    if not value:
        return None

    v = value.strip().lower()

    homme_terms = ["homme", "masculin", "m", "h", "mâle"]
    femme_terms = ["femme", "feminin", "féminin", "fille", "f"]

    if v in homme_terms:
        return "Homme"
    if v in femme_terms:
        return "Femme"

    return None

In [31]:
sexes = [(s, normalize_sex(s)) for s in profiles['sex'].unique()]
for s in sexes:
    print(s)

('Femme', 'Femme')
('Homme', 'Homme')
('Masculin', 'Homme')
(None, None)
('Fille', 'Femme')
('F', 'Femme')
('M', 'Homme')
('H', 'Homme')
('Féminin', 'Femme')
('Feminin', 'Femme')
('Mâle', 'Homme')


## Country of residence

In [39]:
profiles["country_of_residence_str"] = profiles["country_of_residence"].apply(
    lambda x: ", ".join(x) if isinstance(x, (list, np.ndarray)) else x
)
profiles['country_of_residence_str'].unique()

array(['Canada', 'France', 'France, IDF', 'Canada, USA, Europe, Cameroun',
       'Cameroun', '', 'Canada, USA', 'Belgique',
       'Belgique, Luxembourg, France, Allemagne, UE',
       'France, Île-de-France', 'USA', 'Suisse', 'Europe',
       'France, Île-de-France, UE', 'France, Europe',
       'Canada, États-Unis', 'Germany', 'Allemagne',
       'Allemagne, Cameroun', 'France, Union européenne',
       'Amérique du Nord', 'Italie', 'UE', 'Canada, USA, UE',
       'Angleterre', 'Cameroun, Étranger', 'Cameroon',
       'Europe, Amérique, Cameroun', 'Allemagne, Europe',
       'Allemagne, Suisse, Union Européenne, Canada, USA', 'Gabon',
       'England', 'England, Europe', 'France, Union Européenne',
       'Europe, Amérique', 'Canada, Angleterre, Allemagne', <NA>,
       'Canada, USA, Europe, Australie, Asie',
       'France, Belgique, Italie, Canada, Suisse', "Côte d'Ivoire",
       'Amerique du nord', 'Angleterre, Allemagne, Canada',
       'Europe, France', 'UE, Outre-Atlantique',

In [59]:
profiles['country_of_origin'].unique()

array(['C', None, 'Cameroun', 'Cameroon', 'Cambodge'], dtype=object)

In [70]:
country_map = {
    "france": "France",
    "idf": "France",
    "ile-de-france": "France",
    "union europeenne": "Union Européenne",
    "ue": "Union Européenne",
    "eu": "Union Européenne",
    "europe": "Union Européenne",
    "belgique": "Belgique",
    "luxembourg": "Luxembourg",
    "suisse": "Suisse",
    "allemagne": "Allemagne",
    "germany": "Allemagne",
    "uk": "Angleterre",
    "england": "Angleterre",
    "angleterre": "Angleterre",
    "espagne": "Espagne",
    "italie": "Italie",
    "etats-unis": "États-Unis",
    "usa": "États-Unis",
    "amerique du nord": "États-Unis",
    "canada": "Canada",
    "cameroun": "Cameroun",
    "cameroon": "Cameroun",
    "gabon": "Gabon",
    "senegal": "Sénégal",
    "republique centrafricaine": "République Centrafricaine",
    "central african republic": "République Centrafricaine",
    "asie": "Asie",
    "afrique": "Afrique",
    "afrique de l’ouest": "Afrique",
    "amerique": "Amérique",
    "schengen": "Union Européenne",
    "espace schengen": "Union Européenne"
}

# 🔹 Dictionnaire de correspondance pays → continent
continent_map = {
    "France": "Europe",
    "Belgique": "Europe",
    "Luxembourg": "Europe",
    "Suisse": "Europe",
    "Allemagne": "Europe",
    "Union Européenne": "Europe",
    "Angleterre": "Europe",
    "Espagne": "Europe",
    "Italie": "Europe",
    "Canada": "Amérique du Nord",
    "États-Unis": "Amérique du Nord",
    "Cameroun": "Afrique",
    "Gabon": "Afrique",
    "Sénégal": "Afrique",
    "République Centrafricaine": "Afrique",
    "Afrique": "Afrique",
    "Asie": "Asie",
    "Amérique": "Amérique du Nord"
}
import numpy as np
import pandas as pd
import unidecode

def normalize_and_map_country(value):
    """
    Nettoie, normalise et mappe un ou plusieurs pays à leurs continents.
    Retourne une liste de dicts : [{"country": ..., "continent": ...}]
    """
    # Si c'est un array, une Series ou une liste, on transforme en string
    if isinstance(value, (list, np.ndarray, pd.Series)):
        value = ", ".join(str(v) for v in value if pd.notna(v))

    # Vérifier NaN ou chaine vide
    if pd.isna(value) or not isinstance(value, str) or value.strip() == "":
        return []

    text = unidecode.unidecode(value.strip().lower())

    parts = [p.strip() for p in text.split(",") if p.strip()]

    result = []
    for p in parts:
        normalized_country = country_map.get(p, p.capitalize())
        continent = continent_map.get(normalized_country, "Autre")
        result.append({
            "country": normalized_country,
            "continent": continent
        })
    return result


In [71]:
profiles["countries_info"] = profiles["country_of_origin"].apply(normalize_and_map_country)
profiles_exploded = profiles.explode("countries_info", ignore_index=True)

# Extraire chaque champ pour faciliter les analyses
profiles_exploded["country"] = profiles_exploded["countries_info"].apply(
    lambda x: x["country"] if isinstance(x, dict) else None
)
profiles_exploded["continent"] = profiles_exploded["countries_info"].apply(
    lambda x: x["continent"] if isinstance(x, dict) else None
)

profiles_exploded.drop(columns=["countries_info"], inplace=True)

In [72]:
profiles_exploded[['country_of_residence', 'country_of_origin', 'country', 'continent']]

,country_of_residence,country_of_origin,country,continent
0,[Canada],C,C,Autre
1,[Canada],None,None,None
2,[France],None,None,None
3,"[France, IDF]",None,None,None
4,[Canada],C,C,Autre
...,...,...,...,...
1213,[],None,None,None
1214,[],None,None,None
1215,[],None,None,None
1216,[Cameroun],Cameroun,Cameroun,Afrique


In [83]:
profiles["countries_info"] = profiles[profiles['profile_type'] == 'ADVERTISER']["country_of_residence"].apply(normalize_and_map_country)
profiles_exploded = profiles.explode("countries_info", ignore_index=True)

# Extraire chaque champ pour faciliter les analyses
profiles_exploded["country"] = profiles_exploded["countries_info"].apply(
    lambda x: x["country"] if isinstance(x, dict) else None
)
profiles_exploded["continent"] = profiles_exploded["countries_info"].apply(
    lambda x: x["continent"] if isinstance(x, dict) else None
)

profiles_exploded.drop(columns=["countries_info"], inplace=True)

In [84]:
profiles_exploded[['country_of_residence', 'country', 'continent']]

,country_of_residence,country,continent
0,[Canada],Canada,Amérique du Nord
1,[Canada],None,None
2,[France],France,Europe
3,"[France, IDF]",None,None
4,[Canada],Canada,Amérique du Nord
...,...,...,...
1223,[],None,None
1224,[],None,None
1225,[],None,None
1226,[Cameroun],Cameroun,Afrique


In [85]:
for i, p in profiles_exploded.iterrows():
    print(f"{p['country_of_residence']} -- {p['country']} -- {p['continent']}")

['Canada'] -- Canada -- Amérique du Nord
['Canada'] -- None -- None
['France'] -- France -- Europe
['France' 'IDF'] -- None -- None
['Canada'] -- Canada -- Amérique du Nord
['Canada'] -- None -- None
['France'] -- France -- Europe
['France'] -- None -- None
['Canada'] -- Canada -- Amérique du Nord
['Canada' 'USA' 'Europe' 'Cameroun'] -- None -- None
['Cameroun'] -- Cameroun -- Afrique
['Canada'] -- None -- None
['Cameroun'] -- Cameroun -- Afrique
[] -- None -- None
[] -- None -- None
[] -- None -- None
['Cameroun'] -- Cameroun -- Afrique
['France'] -- None -- None
['France'] -- France -- Europe
['France'] -- None -- None
['Canada'] -- Canada -- Amérique du Nord
['Canada' 'USA'] -- None -- None
['Belgique'] -- Belgique -- Europe
['Belgique'] -- None -- None
['Cameroun'] -- Cameroun -- Afrique
['Cameroun'] -- None -- None
['Canada'] -- Canada -- Amérique du Nord
['Canada'] -- None -- None
['Canada'] -- Canada -- Amérique du Nord
['Canada'] -- None -- None
['Canada'] -- Canada -- Amérique

In [86]:
profiles["countries_info"] = profiles[profiles['profile_type'] == 'DESIRED']["country_of_residence"].apply(normalize_and_map_country)
profiles_exploded = profiles.explode("countries_info", ignore_index=True)

# Extraire chaque champ pour faciliter les analyses
profiles_exploded["country"] = profiles_exploded["countries_info"].apply(
    lambda x: x["country"] if isinstance(x, dict) else None
)
profiles_exploded["continent"] = profiles_exploded["countries_info"].apply(
    lambda x: x["continent"] if isinstance(x, dict) else None
)

profiles_exploded.drop(columns=["countries_info"], inplace=True)

In [87]:
for i, p in profiles_exploded.iterrows():
    print(f"{p['country_of_residence']} -- {p['country']} -- {p['continent']}")

['Canada'] -- None -- None
['Canada'] -- Canada -- Amérique du Nord
['France'] -- None -- None
['France' 'IDF'] -- France -- Europe
['France' 'IDF'] -- France -- Europe
['Canada'] -- None -- None
['Canada'] -- Canada -- Amérique du Nord
['France'] -- None -- None
['France'] -- France -- Europe
['Canada'] -- None -- None
['Canada' 'USA' 'Europe' 'Cameroun'] -- Canada -- Amérique du Nord
['Canada' 'USA' 'Europe' 'Cameroun'] -- États-Unis -- Amérique du Nord
['Canada' 'USA' 'Europe' 'Cameroun'] -- Union Européenne -- Europe
['Canada' 'USA' 'Europe' 'Cameroun'] -- Cameroun -- Afrique
['Cameroun'] -- None -- None
['Canada'] -- Canada -- Amérique du Nord
['Cameroun'] -- None -- None
[] -- None -- None
[] -- None -- None
[] -- None -- None
['Cameroun'] -- None -- None
['France'] -- France -- Europe
['France'] -- None -- None
['France'] -- France -- Europe
['Canada'] -- None -- None
['Canada' 'USA'] -- Canada -- Amérique du Nord
['Canada' 'USA'] -- États-Unis -- Amérique du Nord
['Belgique'] -

PAS DE DECISION A PRENDRE POUR LE MOMENT, IL FAUDRAIT MIEUX APPRECIER LES RESULTATS EN LES CONFRONTANT AVEC LE ADS

## Region of origin

In [77]:
profiles['region_of_origin'].unique()

array([None, 'O', 'R', 'Î', 'o', 'A', 'S', 'Ouest Cameroun', 'Ouest',
       'Centre', 'Yaoundé', 'Douala', 'Littoral', 'Nord Cameroun', 'West',
       'Ouest (NDÉ)', 'Menoua', 'Koung-Khi', 'Bassa',
       "L'ouest du Cameroun", 'RESSE 14', 'Hauts Plateaux', 'Mboa',
       'Benguit', 'Île-de-France', 'Bamboutos', 'Mbam', 'Dla', 'l ouest',
       'SUD', 'Rennes', "Bassa'a", 'Haut-Nkam', 'NDE', "L'ouest",
       'Bagangté', 'Région parisienne', 'kho', 'Ouest du Cameroun', 'Kho',
       'Sud', 'Adamaoua', 'Bangangte', 'ouest Cameroun', 'Bafoussam',
       'Ouest-Cameroun', "L'Ouest", 'Nord', 'Yde', "l'ouest", 'ouest',
       'Est', 'Ndé', 'Dschang', 'littoral', 'Nord-Ouest',
       'Est du Cameroun'], dtype=object)

In [79]:
profiles['region_of_origin'].value_counts(normalize=True) * 100

region_of_origin
Ouest                  54.545455
Douala                  6.684492
Centre                  5.882353
Ouest Cameroun          5.080214
O                       4.278075
Yaoundé                 3.475936
Littoral                2.406417
Menoua                  1.336898
Haut-Nkam               0.802139
Île-de-France           0.802139
Hauts Plateaux          0.802139
Bassa'a                 0.534759
ouest Cameroun          0.534759
Bamboutos               0.534759
Koung-Khi               0.534759
Bassa                   0.534759
ouest                   0.534759
o                       0.534759
Bafoussam               0.267380
Kho                     0.267380
Sud                     0.267380
Nord-Ouest              0.267380
Adamaoua                0.267380
Bangangte               0.267380
littoral                0.267380
Ouest-Cameroun          0.267380
kho                     0.267380
Dschang                 0.267380
Ndé                     0.267380
L'Ouest                 0.

In [131]:
region_map = {
    # Ouest
    "ouest": "Ouest",
    "ouest cameroun": "Ouest",
    "ouest-cameroun": "Ouest",
    "ouest (ndé)": "Ouest",
    "l'ouest": "Ouest",
    "l ouest": "Ouest",
    "l'ouest du cameroun": "Ouest",
    "ouest du cameroun": "Ouest",
    "o": "Ouest",
    "ndé": "Ouest",
    "nde": "Ouest",
    "bafoussam": "Ouest",
    "dschang": "Ouest",
    "menoua": "Ouest",
    "koung-khi": "Ouest",
    "bamboutos": "Ouest",
    "bangangte": "Ouest",
    "bagangté": "Ouest",
    "bandjoun": "Ouest",
    "balatchi": "Ouest",
    "mbouda": "Ouest",
    "bamendjou": "Ouest",
    "bayangam": "Ouest",
    "mbamboutos": "Ouest",
    "haut-nkam": "Ouest",
    "haut nkam": "Ouest",
    "hauts plateaux": "Ouest",
    "mboa": "Centre",  # village de Mbam-Mfoundi
    "eton": "Centre",
    "bassa'a": "Littoral",
    "bassa": "Littoral",
    "nko": "Ouest",

    # Centre
    "centre": "Centre",
    "yaoundé": "Centre",
    "mbam": "Centre",
    "yde": "Centre",  # abréviation pour Yaoundé
    "okola": "Centre",

    # Littoral
    "littoral": "Littoral",
    "douala": "Littoral",

    # Nord
    "nord": "Nord",
    "nord cameroun": "Nord",
    "adamaoua": "Adamaoua",  # Adamawa
    "est": "Est",
    "est du cameroun": "Est",

    # Sud
    "sud": "Sud",
    "sud-ouest": "Sud-Ouest",
    "sud ouest": "Sud-Ouest",

    # Nord-Ouest
    "nord-ouest": "Nord-Ouest",
    "north-west": "Nord-Ouest",

    # Sud-Ouest
    "south-west": "Sud-Ouest",

    # Autres ou hors Cameroun
    "île-de-france": None,
    "rennes": None,
    "région parisienne": None,
    "west": "Ouest",  # parfois utilisé par anglophones
    "kho": "Ouest",
    "dla": "Ouest",
    "resse 14": None,
    "a": None,
    "s": None,
    "r": None,
    "cameroon": None,  # général
}

# Supposons profiles['region_of_origin'] et profiles['village_of_origin'] existent
def normalize_region(value):
    if pd.isna(value):
        return None
    key = str(value).strip().lower()
    return region_map.get(key, None)  # retourne None si valeur inconnue

# Normaliser les régions
profiles['region_normalized'] = profiles['region_of_origin'].apply(normalize_region)

# Si la région est manquante mais le village est connu, utiliser le mapping village -> région
def fill_region_from_village(row):
    if row['region_normalized'] is None and pd.notna(row['village_of_origin']):
        key = str(row['village_of_origin']).strip().lower()
        return region_map.get(key, None)
    return row['region_normalized']

profiles['region_normalized'] = profiles.apply(fill_region_from_village, axis=1)

# Nettoyer village_of_origin si c'est en réalité une région ou artefact
def clean_village(value):
    if pd.isna(value):
        return None
    key = str(value).strip().lower()
    if key in region_map:
        # Si la clé correspond à une région, on ne garde pas dans village
        return None
    return value

profiles['village_cleaned'] = profiles['village_of_origin'].apply(clean_village)


In [132]:
profiles['region_normalized'] = profiles['region_of_origin'].apply(normalize_region)
profiles['region_normalized'].value_counts(normalize=True) * 100

region_normalized
Ouest         76.584022
Littoral      10.743802
Centre        10.468320
Nord           0.550964
Sud            0.550964
Est            0.550964
Adamaoua       0.275482
Nord-Ouest     0.275482
Name: proportion, dtype: float64

In [235]:
profiles[profiles['profile_type'] == 'ADVERTISER']['region_normalized'].value_counts(normalize=True) * 100

region_normalized
Ouest         78.529412
Littoral      10.000000
Centre         9.117647
Nord           0.588235
Sud            0.588235
Est            0.588235
Adamaoua       0.294118
Nord-Ouest     0.294118
Name: proportion, dtype: float64

In [236]:
profiles[profiles['profile_type'] == 'DESIRED']['region_normalized'].value_counts(normalize=True) * 100

region_normalized
Ouest       47.826087
Centre      30.434783
Littoral    21.739130
Name: proportion, dtype: float64

In [133]:
profiles['region_normalized'].unique()

array([None, 'Ouest', 'Centre', 'Littoral', 'Nord', 'Sud', 'Adamaoua',
       'Est', 'Nord-Ouest'], dtype=object)

## Village of origin

In [118]:
profiles['village_of_origin'].value_counts(normalize=True)*100

village_of_origin
Bandjoun      16.666667
Bagangté       6.666667
Yaoundé        6.666667
kho            6.666667
Bafou          6.666667
Kho            6.666667
Bassa'a        6.666667
Mbouda         6.666667
Dschang        3.333333
Balatchi       3.333333
Bangangté      3.333333
Basa’a         3.333333
Eton           3.333333
Bamendjou      3.333333
Bayangam       3.333333
Mbamboutos     3.333333
Okola          3.333333
Haut Nkam      3.333333
Nko            3.333333
Name: proportion, dtype: float64

In [130]:
for ix, d in profiles[['region_of_origin', 'village_of_origin']].iterrows():
    if d['region_of_origin'] is not None: # and d['village_of_origin'] is not None: 
        print(d['region_of_origin'], " -- ", d['village_of_origin'])

O  --  None
R  --  None
O  --  None
O  --  None
O  --  None
Î  --  None
O  --  None
O  --  None
O  --  None
o  --  None
O  --  None
O  --  None
O  --  None
o  --  Bagangté
A  --  None
O  --  None
O  --  None
S  --  None
O  --  None
O  --  None
O  --  None
O  --  None
Ouest Cameroun  --  None
Ouest  --  None
Centre  --  None
Ouest Cameroun  --  None
Ouest Cameroun  --  None
Ouest  --  None
Ouest  --  None
Ouest Cameroun  --  None
Yaoundé  --  None
Centre  --  Basa’a
Ouest  --  None
Douala  --  None
Littoral  --  None
Douala  --  None
Ouest  --  None
Ouest  --  None
Ouest  --  None
Ouest  --  None
Centre  --  None
Centre  --  None
Yaoundé  --  None
Ouest  --  None
Ouest  --  None
Yaoundé  --  None
Ouest  --  None
Ouest  --  Bayangam
Ouest Cameroun  --  None
Centre  --  Bassa'a
Nord Cameroun  --  None
Ouest  --  None
Ouest  --  None
Ouest Cameroun  --  None
Ouest Cameroun  --  None
Ouest  --  None
Ouest  --  None
West  --  None
Ouest  --  None
Ouest (NDÉ)  --  None
Yaoundé  --  None
Doual

In [134]:
profiles['village_cleaned'].value_counts(normalize=True) * 100

village_cleaned
Bafou        50.0
Basa’a       25.0
Bangangté    25.0
Name: proportion, dtype: float64

## Sector of activity

In [140]:
profiles['sector_of_activity'].value_counts(normalize=True, dropna=False)

sector_of_activity
None                            0.741379
Étudiante                       0.016420
Étudiant                        0.015599
Entrepreneur                    0.012315
Ingénieur                       0.011494
                                  ...   
Vétérinaire                     0.000821
chirurgie                       0.000821
Warboy                          0.000821
Marketing                       0.000821
Organisatrice événementielle    0.000821
Name: proportion, Length: 156, dtype: float64

In [141]:
profiles.columns

Index(['id', 'ad_id', 'profile_type', 'name', 'religion', 'age', 'sex',
       'country_of_residence', 'country_of_origin', 'region_of_origin',
       'village_of_origin', 'sector_of_activity', 'marital_status',
       'has_children', 'number_of_children', 'qualities', 'values', 'defects',
       'interests', 'height', 'weight', 'physical_appearance',
       'economic_situation', 'education_level', 'illness', 'relationship_goal',
       'created_at', 'country_of_residence_str', 'countries_info',
       'region_normalized', 'village_cleaned'],
      dtype='object')

In [142]:
sector_map = {
    # Études / Éducation
    "Étudiant": "Études / Éducation",
    "Étudiante": "Études / Éducation",
    "Enseignant": "Études / Éducation",
    "Professeur": "Études / Éducation",
    
    # Entrepreneuriat / Affaires
    "Entrepreneur": "Entrepreneuriat / Affaires",
    "Commerce": "Entrepreneuriat / Affaires",
    "Business": "Entrepreneuriat / Affaires",
    
    # Ingénierie / Technique / IT
    "Ingénieur": "Ingénierie / Technique / IT",
    "Informaticien": "Ingénierie / Technique / IT",
    "Technicien": "Ingénierie / Technique / IT",
    
    # Médecine / Santé
    "Médecin": "Médecine / Santé",
    "Vétérinaire": "Médecine / Santé",
    "Chirurgie": "Médecine / Santé",
    "Infirmier": "Médecine / Santé",
    
    # Marketing / Communication / Vente
    "Marketing": "Marketing / Communication / Vente",
    "Commercial": "Marketing / Communication / Vente",
    "Communication": "Marketing / Communication / Vente",
    
    # Art / Culture / Evénementiel
    "Organisatrice événementielle": "Art / Culture / Evénementiel",
    "Artiste": "Art / Culture / Evénementiel",
    "Musicien": "Art / Culture / Evénementiel",
    
    # Autres ou non renseigné
    None: "Non renseigné"
}

# Appliquer le mapping
profiles['sector_grouped'] = profiles['sector_of_activity'].map(lambda x: sector_map.get(x, "Autres"))


## Marital status

In [143]:
profiles['marital_status'].value_counts()

marital_status
Célibataire               234
célibataire                36
Divorcée                    3
Celibataire                 2
Divorcé                     2
Non divorcé                 2
Veuve                       2
prêt pour le mariage        1
seul                        1
Célibataire ou divorcé      1
Mariée                      1
Indétectable                1
Séparée                     1
Name: count, dtype: int64

In [144]:
marital_map = {
    # Célibataires
    "Célibataire": "Célibataire",
    "célibataire": "Célibataire",
    "Celibataire": "Célibataire",
    "prêt pour le mariage": "Célibataire",
    "seul": "Célibataire",
    
    # Divorcé(e) / Séparé(e)
    "Divorcée": "Divorcé(e)",
    "Divorcé": "Divorcé(e)",
    "Séparée": "Divorcé(e)",
    "Non divorcé": "Célibataire",  # à vérifier selon le contexte
    "Célibataire ou divorcé": "Divorcé(e)",
    
    # Marié(e)
    "Mariée": "Marié(e)",
    
    # Veuf / Veuve
    "Veuve": "Veuf / Veuve",
    
    # Inconnu / Indétectable
    # "Indétectable": "Inconnu"
}

profiles['marital_status_normalized'] = profiles['marital_status'].map(lambda x: marital_map.get(x, "Autre"))

In [145]:
illness_keywords = [
    "indétectable", "vih", "diabète", "hypertension", "cancer", 
    "asthme", "hiv", "hepatite", "malade", "condition médicale"
]

# Fonction pour détecter si l'annonce mentionne une maladie
def detect_illness(text):
    if text is None:
        return False
    text_lower = str(text).lower()
    return any(keyword in text_lower for keyword in illness_keywords)

# Créer la feature has_illness
profiles['has_illness'] = profiles['illness'].apply(detect_illness) | \
                          profiles['marital_status'].apply(detect_illness)


## Children

In [148]:
profiles[['has_children', 'number_of_children']]

,has_children,number_of_children
0,False,sans enfant
1,False,sans enfant
2,True,1 enfant
3,False,None
4,True,3 enfants
...,...,...
1213,False,None
1214,True,1
1215,False,None
1216,False,0


In [152]:
for index, c in profiles[['has_children', 'number_of_children']].iterrows():
    print(c['has_children'], " --- ", c['number_of_children'])

False  ---  sans enfant
False  ---  sans enfant
True  ---  1 enfant
False  ---  None
True  ---  3 enfants
False  ---  None
False  ---  None
False  ---  None
False  ---  sans enfant
False  ---  0-1 enfant
True  ---  3 enfants
False  ---  None
False  ---  sans enfant
False  ---  sans enfant
False  ---  None
False  ---  None
False  ---  sans enfant
False  ---  sans enfant
False  ---  sans enfant
False  ---  sans enfant
False  ---  sans enfant
False  ---  sans enfant
True  ---  1 enfant
False  ---  None
False  ---  sans enfant
False  ---  sans enfant
False  ---  sans enfant
False  ---  None
True  ---  3 enfants
False  ---  None
False  ---  sans enfant
False  ---  None
True  ---  1 enfant
False  ---  None
False  ---  sans enfant
False  ---  None
False  ---  sans enfant
False  ---  None
False  ---  sans enfant
False  ---  sans enfant
True  ---  1 enfant
False  ---  sans enfant
False  ---  None
False  ---  None
False  ---  None
False  ---  None
False  ---  None
False  ---  None
False  ---  sa

In [153]:
profiles['has_children'].value_counts(normalize=True) * 100

has_children
False    82.758621
True     17.241379
Name: proportion, dtype: float64

In [154]:
profiles['number_of_children'].value_counts(normalize=True) * 100

number_of_children
0              59.756098
1              23.519164
sans enfant     6.620209
2               6.097561
1 enfant        1.219512
3               1.045296
0-1 enfant      0.696864
3 enfants       0.522648
4               0.522648
Name: proportion, dtype: float64

In [161]:
def clean_number_of_children(val):
    if val is None:
        return 0
    val_str = str(val).lower().strip()
    
    # Cas explicites
    if val_str in ["sans enfant", "0"]:
        return 0
    if val_str in ["1 enfant", "1", "0-1 enfant"]:
        return 1
    if val_str in ["2"]:
        return 2
    if val_str in ["3 enfants", "3"]:
        return 3
    if val_str in ["4"]:
        return 4
    
    # Extraire nombre si présent dans le texte
    match = re.search(r"\d+", val_str)
    if match:
        return int(match.group(0))
    
    # Si rien trouvé, considérer 0
    return 0

# Appliquer le nettoyage
profiles['number_of_children_cleaned'] = profiles['number_of_children'].apply(clean_number_of_children)

# Recalculer has_children sur base du nombre d'enfants
profiles['has_children_cleaned'] = profiles['number_of_children_cleaned'].apply(lambda x: x > 0)


In [162]:
profiles['has_children_cleaned'].value_counts(normalize=True) * 100

has_children_cleaned
False    84.154351
True     15.845649
Name: proportion, dtype: float64

In [163]:
profiles['number_of_children_cleaned'].value_counts(normalize=True) * 100

number_of_children_cleaned
0    84.154351
1    11.986864
2     2.873563
3     0.738916
4     0.246305
Name: proportion, dtype: float64

## Qualities, values, defects, interests

In [177]:
profiles[['qualities', 'values', 'defects', 'interests']].head()

,qualities,values,defects,interests
0,"[respectueuse, douce, travailleuse]",[],[],[]
1,"[travailleur, respectueux, sens de l'humour]",[],[],[]
2,"[positive, intègre, joie de vivre contagieuse]","[Générosité, respect, tolérance, famille, comm...",<NA>,"[voyages, développement personnel, plaisirs si..."
3,"[charmant, confiance en toi, attentionné, conc...",[],<NA>,"[sportif, courir]"
4,"[respectueuse, gentille, responsable, travaill...","[respect, vérité, amour, communication, valeur...",[],"[danser, voyager, sorties en couple, sorties e..."


In [176]:
profiles[['qualities', 'values', 'defects', 'interests']].isna().sum()

qualities     4
values       29
defects      38
interests    35
dtype: int64

In [178]:
profiles['qualities'].value_counts()

qualities
[ambitieuse]                                                                                                    5
[sportif]                                                                                                       3
[dominant]                                                                                                      2
[responsable]                                                                                                   2
[]                                                                                                              1
                                                                                                               ..
[]                                                                                                              1
[respectueuse, posé, dynamique]                                                                                 1
[calme, persévérant]                                                          

In [183]:
def list_to_string(lst):
    # Gestion des NaN ou None
    if lst is None:
        return ""

    # Si c’est une série ou un array (ex: numpy), convertir en liste
    if isinstance(lst, (list, tuple, set)):
        items = list(lst)
    elif hasattr(lst, '__iter__') and not isinstance(lst, str):
        items = list(lst)
    else:                       # une seule valeur scalaire
        items = [lst]

    # Nettoyer : retirer valeurs vides, capitaliser et trier, enlever doublons
    cleaned = sorted({
        str(x).strip().capitalize()
        for x in items
        if not pd.isna(x) and str(x).strip() != ""
    })

    return "; ".join(cleaned)

# Application
for col in ['qualities', 'values', 'defects', 'interests']:
    profiles[f"{col}_cleaned"] = profiles[col].apply(list_to_string)

In [184]:
profiles.columns

Index(['id', 'ad_id', 'profile_type', 'name', 'religion', 'age', 'sex',
       'country_of_residence', 'country_of_origin', 'region_of_origin',
       'village_of_origin', 'sector_of_activity', 'marital_status',
       'has_children', 'number_of_children', 'qualities', 'values', 'defects',
       'interests', 'height', 'weight', 'physical_appearance',
       'economic_situation', 'education_level', 'illness', 'relationship_goal',
       'created_at', 'country_of_residence_str', 'countries_info',
       'region_normalized', 'village_cleaned', 'sector_grouped',
       'marital_status_normalized', 'has_illness',
       'number_of_children_cleaned', 'has_children_cleaned',
       'qualities_cleaned', 'values_cleaned', 'defects_cleaned',
       'interests_cleaned'],
      dtype='object')

## Height / Weight

In [185]:
profiles[['height', 'weight']]

,height,weight
0,1.70 m,70 kg
1,1.72 m,None
2,1.68 m,67 kg
3,1.77 m,None
4,None,None
...,...,...
1213,None,None
1214,None,None
1215,None,None
1216,None,None


In [186]:
profiles['height'].value_counts()

height
1.70 m             45
1.80 m             37
1.60 m             37
1.65 m             31
1.75 m             29
1.68 m             21
1.78 m             21
1.76 m             20
1.77 m             19
1.85 m             14
1.67 m             14
1.72 m             13
1.74 m             13
1.69 m             12
1.71 m             12
1.63 m             12
1.73 m             10
1.90 m              8
1.57 m              8
1.55 m              7
1.62 m              7
1.82 m              6
1.79 m              6
1.64 m              6
1.66 m              6
1.86 m              4
1.91 m              4
1.59 m              4
1.84 m              4
1.89 m              4
1.83 m              3
1.61 m              3
1.58 m              2
1.87 m              2
1.93 m              2
1.81 m              2
1.65-1.85 m         2
1.88 m              2
1.80 m+             1
1.70-1.80 m         1
1.95 m              1
1.65-1.90 m         1
plus de 1.68 m      1
1.65-1.72 m         1
1.65 à 1.85 m       1
Moy

In [187]:
profiles['weight'].value_counts()

weight
70 kg         32
80 kg         21
85 kg         17
75 kg         16
78 kg         16
65 kg         13
90 kg         12
60 kg         11
82 kg          7
69 kg          6
72 kg          6
63 kg          6
95 kg          6
55 kg          6
77 kg          4
76 kg          4
68 kg          4
79 kg          4
74 kg          4
57 kg          4
87 kg          4
73 kg          3
83 kg          3
62 kg          3
67 kg          3
58 kg          3
88 kg          3
89 kg          2
93 kg          2
71 kg          2
86 kg          2
54 kg          2
<82 kg         1
59 kg          1
53 kg          1
moyenne        1
84 kg          1
65-90 kg       1
65 à 85 kg     1
65-85 kg       1
98 kg          1
52 kg          1
94 kg          1
105 kg         1
64 kg          1
66 kg          1
Name: count, dtype: int64

In [188]:
def extract_height_range(val):
    if pd.isna(val):
        return np.nan, np.nan, np.nan
    
    val = str(val).lower().strip()
    
    # Intervalle "X-Y" ou "X à Y"
    interval_match = re.findall(r'(\d+\.\d+)\s*(?:-|à)\s*(\d+\.\d+)', val)
    if interval_match:
        a, b = map(float, interval_match[0])
        mean = round((a + b)/2, 2)
        return a, b, mean
    
    # "plus de X" ou ">= X"
    plus_match = re.search(r'(?:plus de|>=)\s*(\d+\.\d+)', val)
    if plus_match:
        x = float(plus_match.group(1))
        return x, np.nan, np.nan
    
    # "moins de X"
    moins_match = re.search(r'(?:moins de)\s*(\d+\.\d+)', val)
    if moins_match:
        x = float(moins_match.group(1))
        return np.nan, x, np.nan
    
    # Valeur simple "X m"
    number_match = re.search(r'\d+\.\d+', val)
    if number_match:
        x = float(number_match.group())
        return x, x, x
    
    return np.nan, np.nan, np.nan

# Appliquer et créer les colonnes
profiles[['height_min', 'height_max', 'height_mean']] = profiles['height'].apply(
    lambda x: pd.Series(extract_height_range(x))
)


In [194]:
profiles[['height_min', 'height_max', 'height_mean']]

,height_min,height_max,height_mean
0,1.70,1.70,1.70
1,1.72,1.72,1.72
2,1.68,1.68,1.68
3,1.77,1.77,1.77
4,NaN,NaN,NaN
...,...,...,...
1213,NaN,NaN,NaN
1214,NaN,NaN,NaN
1215,NaN,NaN,NaN
1216,NaN,NaN,NaN


In [189]:
def extract_weight_range(val):
    if pd.isna(val):
        return np.nan, np.nan, np.nan
    
    val = str(val).lower().strip()
    
    # Intervalle "X-Y" ou "X à Y"
    interval_match = re.findall(r'(\d+)\s*(?:-|à)\s*(\d+)', val)
    if interval_match:
        a, b = map(float, interval_match[0])
        mean = round((a + b)/2, 2)
        return a, b, mean
    
    # "> X" ou "plus de X"
    plus_match = re.search(r'(?:plus de|>)\s*(\d+)', val)
    if plus_match:
        x = float(plus_match.group(1))
        return x, np.nan, np.nan
    
    # "< X" ou "moins de X"
    moins_match = re.search(r'(?:moins de|<)\s*(\d+)', val)
    if moins_match:
        x = float(moins_match.group(1))
        return np.nan, x, np.nan
    
    # Valeur simple "X kg"
    number_match = re.search(r'\d+', val)
    if number_match:
        x = float(number_match.group())
        return x, x, x
    
    # "moyenne" ou autre texte
    return np.nan, np.nan, np.nan

# Appliquer et créer les colonnes
profiles[['weight_min', 'weight_max', 'weight_mean']] = profiles['weight'].apply(
    lambda x: pd.Series(extract_weight_range(x))
)


In [193]:
profiles[['weight_min', 'weight_max', 'weight_mean']]

,weight_min,weight_max,weight_mean
0,70.0,70.0,70.0
1,NaN,NaN,NaN
2,67.0,67.0,67.0
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1213,NaN,NaN,NaN
1214,NaN,NaN,NaN
1215,NaN,NaN,NaN
1216,NaN,NaN,NaN


In [201]:
[(h, ) + extract_weight_range(h) for h in profiles['weight']]

[('70 kg', 70.0, 70.0, 70.0),
 (None, nan, nan, nan),
 ('67 kg', 67.0, 67.0, 67.0),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 ('78 kg', 78.0, 78.0, 78.0),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 ('78 kg', 78.0, 78.0, 78.0),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 ('60 kg', 60.0, 60.0, 60.0),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 ('78 kg', 78.0, 78.0, 78.0),
 (None, nan, nan, nan),
 ('71 kg', 71.0, 71.0, 71.0),
 (None, nan, nan, nan)

In [203]:
[(h, ) + extract_height_range(h) for h in profiles['height']]

[('1.70 m', 1.7, 1.7, 1.7),
 ('1.72 m', 1.72, 1.72, 1.72),
 ('1.68 m', 1.68, 1.68, 1.68),
 ('1.77 m', 1.77, 1.77, 1.77),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 ('1.77 m', 1.77, 1.77, 1.77),
 (None, nan, nan, nan),
 ('1.75 m', 1.75, 1.75, 1.75),
 ('1.80 m', 1.8, 1.8, 1.8),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 ('1.77 m', 1.77, 1.77, 1.77),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 (None, nan, nan, nan),
 ('1.70 m', 1.7, 1.7, 1.7),
 ('1.77 m', 1.77, 1.77, 1.77),
 (None, nan, nan, nan),
 ('1.80 m', 1.8, 1.8, 1.8),

## Physical appearance

In [207]:
profiles['physical_appearance'].unique()

array([None, 'charmante',
       'corpulence moyenne avec des formes là où il faut',
       'Naturelle et sans artifice',
       'S, a, n, s,  , m, a, q, u, i, l, l, a, g, e',
       'bien bâti et sportif', 'G, r, a, n, d',
       'Teint chocolat, rondeurs',
       'c, o, r, p, u, l, e, n, c, e,  , m, o, y, e, n, n, e',
       'B, e, l, l, e, ,,  , c, o, u, r, b, e, s,  , a, f, r, i, c, a, i, n, e, s, ,,  , p, a, s,  , m, i, n, c, e, ,,  , p, a, s,  , r, o, n, d, e',
       'teint caramel', 'formes, beau visage',
       'b, e, l, l, e, s,  , c, o, u, r, b, e, s,  , a, f, r, i, c, a, i, n, e, s',
       'beautiful', 'corpulence fine',
       'physique de guerrier des temps anciens',
       'p, h, y, s, i, q, u, e,  , d, e,  , g, u, e, r, r, i, e, r,  , d, e, s,  , t, e, m, p, s,  , a, n, c, i, e, n, s',
       'sportif, prend bien soin de son apparence',
       'b, e, l, l, e, s,  , f, o, r, m, e, s,  , a, f, r, i, c, a, i, n, e, s',
       'p, h, y, s, i, q, u, e,  , m, o, y, e, n, ,, 

In [215]:
import pandas as pd
import re
import unidecode

def clean_physical_appearance(val):
    if val is None or pd.isna(val):
        return ""
    
    val = str(val).strip()
    
    # supprimer toutes les lettres séparées par des virgules et/ou espaces
    # ex: 'c, o, r, p, u, l, e' -> 'corpule'
    val = re.sub(r'([a-zA-Z])\s*,\s*', r'\1', val)  # remplace chaque lettre suivie de virgule par la lettre seule
    
    # mettre en minuscules
    val = val.lower()
    
    # retirer accents
    val = unidecode.unidecode(val)
    
    # supprimer espaces multiples
    val = re.sub(r'\s+', ' ', val).strip()
    
    return val

# appliquer
profiles['physical_appearance_cleaned'] = profiles['physical_appearance'].apply(clean_physical_appearance)

In [216]:
profiles['physical_appearance_cleaned'].unique()

array(['', 'charmante',
       'corpulence moyenne avec des formes la ou il faut',
       'naturelle et sans artifice', 'sans, maquillage',
       'bien bati et sportif', 'grand', 'teint chocolatrondeurs',
       'corpulence, moyenne',
       'belle,, , courbes, africaines,, , pas, mince,, , pas, ronde',
       'teint caramel', 'formesbeau visage',
       'belles, courbes, africaines', 'beautiful', 'corpulence fine',
       'physique de guerrier des temps anciens',
       'physique, de, guerrier, des, temps, anciens',
       'sportifprend bien soin de son apparence',
       'belles, formes, africaines',
       'physique, moyen,, , athle, tique, ou, mince', 'claire de peau',
       'fineteint noir', 'teint clair', 'ronde', 'teint noirpetite',
       'moyenne', 'physique de mannequin', 'physique, de, mannequin',
       'petite de corps', 'teint noir',
       'teint clairjovialechaleureuseaccueillante', 'corpulence moyenne',
       'bruneteint naturel', 'jolie', 'teint chocolat',
       '

## Economic Situation

In [219]:
profiles['economic_situation'].unique()

array(['Début de carrière', None, 'stable', 'Stable', 'Emploi stable',
       'Salariée', 'entrepreneur', 'étudiant',
       'situation financière stable', 'boulot bien payé', 'emploi stable',
       'Situation stable', 'Étudiant', 'Emploi', 'Financièrement stable',
       'pas stable financièrement', 'Financièrement posé',
       'non financièrement stable', 'Très bien', 'Entrepreneur',
       '[métier stable]', 'Travailleur', 'fonctionnaire A2',
       'indépendant', 'emploi plus ou moins stable',
       'Stable financièrement', 'Chômage', 'stabilité professionnelle',
       'stable financièrement', 'financièrement stable',
       'Professionnellement actif', 'stable ou en fin de formation',
       'cadre salarié', 'stable financierement', 'Instable', 'Minimale',
       "En recherche d'emploi", "Agent de l'Etat", 'indépendante',
       'financièrement pas à plaindre',
       '[stable financièrement, en fin de formation]',
       'salariée du secteur public', 'Financière stable',
    

In [220]:
def prepare_for_embedding(val):
    """
    Nettoie une valeur textuelle pour la rendre prête pour un embedding :
    - Met en minuscules
    - Supprime accents
    - Supprime crochets et caractères inutiles
    - Remplace multiples espaces par un seul
    - Conserve le reste du texte
    """
    if val is None or pd.isna(val):
        return ""
    
    val = str(val).strip()                    # retirer espaces début/fin
    val = unidecode.unidecode(val)           # retirer accents
    val = val.lower()                         # tout en minuscules
    val = re.sub(r'[\[\]]', '', val)         # supprimer crochets
    val = re.sub(r'\s+', ' ', val)           # remplacer multiples espaces par un seul
    
    return val

# Exemple d'application sur une colonne
profiles['economic_cleaned'] = profiles['economic_situation'].apply(prepare_for_embedding)


In [221]:
profiles['economic_cleaned'].unique()

array(['debut de carriere', '', 'stable', 'emploi stable', 'salariee',
       'entrepreneur', 'etudiant', 'situation financiere stable',
       'boulot bien paye', 'situation stable', 'emploi',
       'financierement stable', 'pas stable financierement',
       'financierement pose', 'non financierement stable', 'tres bien',
       'metier stable', 'travailleur', 'fonctionnaire a2', 'independant',
       'emploi plus ou moins stable', 'stable financierement', 'chomage',
       'stabilite professionnelle', 'professionnellement actif',
       'stable ou en fin de formation', 'cadre salarie', 'instable',
       'minimale', "en recherche d'emploi", "agent de l'etat",
       'independante', 'financierement pas a plaindre',
       'stable financierement, en fin de formation',
       'salariee du secteur public', 'financiere stable',
       'paye ses factures a temps', 'etudiant en fin de formation',
       "cadre de l'administration", 'financierement autonome',
       'autonomie financiere',

## Education level

In [223]:
profiles['education_level'].unique()

array([None, 'Études supérieures', 'Doctorant', 'Bac+3', 'Ingénieur',
       'bac + 3 et plus', 'Baccalauréat', 'étudiant', 'Diplômé',
       'Master 1', 'Professionnel', 'En formation de soins infirmiers',
       'En cours de spécialisation', 'Étudiant', 'Bac +3',
       "['minimum d\\'études']", 'Master', 'bac +4',
       '[études supérieures]', 'formée', 'master 2',
       'Diplôme supérieur de second cycle',
       'Diplômé d’une école de formation de la place', 'Licence',
       'Maternelle', 'Licence professionnelle', 'Apprenti agronome',
       'Licence en commerce', "fin d'études académique", 'Master 2',
       'Médecin', 'fin de formation',
       'Étudiante en 4ème année comptabilité', 'études supérieures',
       'Ingénierie', 'étudiante en audit et contrôle de gestion',
       'Licence pro en marketing', 'Fin d’études universitaires',
       'Étudiant(e)', 'Doctorat', 'En cours de formation',
       '[spécialisation en chirurgie]', 'Vétérinaire', 'bac+4',
       'formation'

In [224]:
def clean_text_for_embedding(text):
    """
    Nettoie les valeurs textuelles en préparation pour les embeddings :
    - Convertit en chaîne de caractères
    - Met en minuscules
    - Supprime accents et ponctuation légère
    - Supprime crochets, guillemets, caractères spéciaux
    - Supprime espaces multiples
    """
    if text is None or pd.isna(text):
        return ""
    
    # Convertir en texte
    text = str(text).strip()
    
    # Supprimer accents
    text = unidecode.unidecode(text)
    
    # Minuscule
    text = text.lower()
    
    # Supprimer crochets, guillemets, doubles quotes, etc.
    text = re.sub(r"[\[\]'\"’]", "", text)
    
    # Supprimer les caractères non alphabétiques de remplissage (ex: \\)
    text = re.sub(r"\\", "", text)
    
    # Supprimer les doubles espaces et ponctuations excessives
    text = re.sub(r"\s+", " ", text)
    
    # Supprimer ponctuations inutiles en bout de phrase
    text = re.sub(r"[.,;:!?-]+$", "", text)
    
    return text.strip()


In [225]:
profiles['education_level_cleaned'] = profiles['education_level'].apply(clean_text_for_embedding)
profiles['education_level_cleaned'].unique()

array(['', 'etudes superieures', 'doctorant', 'bac+3', 'ingenieur',
       'bac + 3 et plus', 'baccalaureat', 'etudiant', 'diplome',
       'master 1', 'professionnel', 'en formation de soins infirmiers',
       'en cours de specialisation', 'bac +3', 'minimum detudes',
       'master', 'bac +4', 'formee', 'master 2',
       'diplome superieur de second cycle',
       'diplome dune ecole de formation de la place', 'licence',
       'maternelle', 'licence professionnelle', 'apprenti agronome',
       'licence en commerce', 'fin detudes academique', 'medecin',
       'fin de formation', 'etudiante en 4eme annee comptabilite',
       'ingenierie', 'etudiante en audit et controle de gestion',
       'licence pro en marketing', 'fin detudes universitaires',
       'etudiant(e)', 'doctorat', 'en cours de formation',
       'specialisation en chirurgie', 'veterinaire', 'bac+4', 'formation',
       'etudiante', 'en formation', 'bac +5', 'licence et master',
       'fin detude (ou de formation)

## Illness

In [228]:
profiles['illness'].value_counts()

illness
Séropositive                  2
[]                            2
handicap                      1
Seropositive                  1
charge virale indétectable    1
Séropositif                   1
séropositive                  1
Name: count, dtype: int64

In [229]:
def normalize_illness(value):
    if value is None or pd.isna(value):
        return "Aucune"
    
    # Nettoyage de base
    value = str(value).strip()
    value = unidecode.unidecode(value.lower())
    value = re.sub(r"[\[\]']", "", value)
    
    # Cas vides ou neutres
    if value in ["", "none", "aucune", "aucun", "[]"]:
        return "Aucune"
    
    # Cas spécifiques
    if "handicap" in value:
        return "Handicap"
    
    if "sero" in value or "vih" in value or "charge virale" in value:
        return "VIH / Séropositif"
    
    # Par défaut
    return "Autre"


In [230]:
profiles['illness_cleaned'] = profiles['illness'].apply(normalize_illness)
profiles['illness_cleaned'].value_counts()

illness_cleaned
Aucune               1211
VIH / Séropositif       6
Handicap                1
Name: count, dtype: int64

## Relationship goal

In [232]:
profiles['relationship_goal'].unique()

array(['fonder une famille solide', None, 'futur époux', 'partenaire',
       'Mari', 'long terme', 'fonder une famille', 'Amour', 'Famille',
       'Coéquipier de la vie', 'relation sérieuse en vue du mariage',
       'sincère', "à la recherche d'une femme de sa vie",
       'prête à s’engager', 'famille', 'relation', 'respect mutuel',
       'relation stable', 'relation sérieuse', 'Partenariat durable',
       'relation sérieuse et sincère', 'Célibataire', 'Sérieux',
       'prête à fonder son foyer', 'intègre', 'Soutien dans mes affaires',
       'Amie', 'prête à vivre en couple', 'relation sincère et pérenne',
       'Relation sérieuse', 'Mariage', 'soumise', 'respectueuse',
       'responsable', 'Future épouse', 'complice',
       'relation sérieuse pouvant aboutir au mariage',
       'rencontre physique réelle', 'cherche une femme', 'relation saine',
       "à la recherche d'un partenaire", 'relation amoureuse sérieuse',
       'fonder notre famille', 'prête à fonder une famille'

In [233]:
profiles['relationship_goal_cleaned'] = profiles['relationship_goal'].apply(clean_text_for_embedding)
profiles['relationship_goal_cleaned'].unique()

array(['fonder une famille solide', '', 'futur epoux', 'partenaire',
       'mari', 'long terme', 'fonder une famille', 'amour', 'famille',
       'coequipier de la vie', 'relation serieuse en vue du mariage',
       'sincere', 'a la recherche dune femme de sa vie',
       'prete a sengager', 'relation', 'respect mutuel',
       'relation stable', 'relation serieuse', 'partenariat durable',
       'relation serieuse et sincere', 'celibataire', 'serieux',
       'prete a fonder son foyer', 'integre', 'soutien dans mes affaires',
       'amie', 'prete a vivre en couple', 'relation sincere et perenne',
       'mariage', 'soumise', 'respectueuse', 'responsable',
       'future epouse', 'complice',
       'relation serieuse pouvant aboutir au mariage',
       'rencontre physique reelle', 'cherche une femme', 'relation saine',
       'a la recherche dun partenaire', 'relation amoureuse serieuse',
       'fonder notre famille', 'prete a fonder une famille',
       'amour veritable', 'relation

Index(['id', 'ad_id', 'profile_type', 'name', 'religion', 'age', 'sex',
       'country_of_residence', 'country_of_origin', 'region_of_origin',
       'village_of_origin', 'sector_of_activity', 'marital_status',
       'has_children', 'number_of_children', 'qualities', 'values', 'defects',
       'interests', 'height', 'weight', 'physical_appearance',
       'economic_situation', 'education_level', 'illness', 'relationship_goal',
       'created_at', 'country_of_residence_str', 'countries_info',
       'region_normalized', 'village_cleaned', 'sector_grouped',
       'marital_status_normalized', 'has_illness',
       'number_of_children_cleaned', 'has_children_cleaned',
       'qualities_cleaned', 'values_cleaned', 'defects_cleaned',
       'interests_cleaned', 'height_min', 'height_max', 'height_mean',
       'weight_min', 'weight_max', 'weight_mean',
       'physical_appearance_cleaned', 'economic_cleaned',
       'education_level_cleaned', 'illness_cleaned',
       'relationship_goal